4. 스레드 동기화

1) 스레드의 문제점
    - Thread를 사용하면 발생할 수 있는 문제가 있다.
      스레드는 모든 자원을 공유한다라고 앞서 말한바 있다.
      예를 들어서 스레드는 전역변수를 전부 공유 하는데
      서로 다른 스레드가 동시에 같은 전역변수에 접근하면 
      원치 않은 결과를 초래할 수가 있다.

In [ ]:
import threading

totalCount = 0 # 전역 변수 선언
class CounterThread(threading.Thread):
    
    def __init__(self):
        super().__init__()
        
    def run(self):
        global totalCount
        for _ in range(2500000):
            totalCount += 1
        print("2,500,000번 카운팅 끝")
        
if __name__ == "__main__":
    for _ in range(4):
        cnt = CounterThread()
        cnt.start()
        
    print("모든 Thread들이 종료될 때까지 기다린다.")
    mainThread = threading.currentThread()
    for thread in threading.enumerate():
        # Main Thread를 제외한 모든 Thread들이 카운팅을 완료하고 끝날 떄 까지 
        # 기다린다.
        if thread is not mainThread:
            thread.join()
    print(f"totalCount = {totalCount}")
    
# 강의 totalCount = 5148973

모든 Thread들이 종료될 때까지 기다린다.
2,500,000번 카운팅 끝
2,500,000번 카운팅 끝
2,500,000번 카운팅 끝2,500,000번 카운팅 끝



1) 스레드의 문제점
    - 위 코드를 실행하면 4개의 Thread가 totalCount 변수에 2,500,000번
      더했으니 4 * 2,500,000번인 10,000,000이 나와야 정상이다.
      하지만 결과는 다르다.
      왜 그럴까?
      바로 totalCount 변수에 동시에 접근을 했기 때문인 것이다.
      
      이런 문제를 해결하고자 한다면 
      스레드 동기화 처리를 해주어야 한다는 것이다.

2) 스레드 동기화 처리
    - 동기화 하는 방법 중에 Lock을 살펴보자.
      Lock은 특정 스레드에서 변수를 사용하기 시작했으면 
      다른 스레드가 사용하지 못하도록 막는 역할을 한다.
      마치 변수를 잠구는 것과 같아서 Lock이라고 부른다.
      변수를 다 사용했으면 그 스레드는 변수에 대한 Lock을 풀어 줘야 한다.
      잠금을 푸는 것은 Release라고 부른다.
      이런 간단한 잠금으로 변수를 서로 다른 스레드가 동시에 접근하지
      못하게 막으면 위의 원치 않는 상황은 해결할 수 있다.
      
    - Lock 사용은 아래와 같이 한다.
        - Lock.aquire() = 잠금 - 다른 스레드들은 접근 못하게 막는다.
          여기 안에 code들은 무조건 한 스레드에 의해서 순차적으로 실행
        - Lock.release() = 잠금해제 - 다른 스레드에게 접근가능 잠금해제
        
    - 통상 스레드는 모든 자원을 공유하기 때문에 원하는 결과값을 얻기 
      위해서는 속도는 조금 느려도 위와 같이 Lock을 이용하여 다른 스레드가
      접근하여 데이터를 훼손하는 것보다 무엇보다 데이터의 신뢰성이
      우선이기 떄문에 Lock을 이용하는 것이다.

tip 임계영역 : 둘 이상의 스레드가 동시에 접근해서는 안되는 공유 자원을
              접근하는 코드를 지칭

In [ ]:
import threading
totalCount = None

# 공유된 변수를 위한 클래스
class ThreadVariable:
    def __init__(self):
        self.lock = threading.Lock()
        self.lockedValue = 0
    
    # 한 Thread만 접근할 수 있도록 설정한다
    def plus(self, value):
        # Lock해서 다른 Thread는 기다리게 만든다.
        self.lock.acquire()
        self.lockedValue += value
        # Lock을 해제해서 다른 Thread도 사용할 수 있도록 만든다.
        self.lock.release()


class CounterThread(threading.Thread):
    def __init__(self):
        super().__init__()
    
    def run(self):
        global totalCount
        for _ in range(2500000):
            totalCount.plus(1)
        print('2,500,000번 카운팅 끝!')

        
if __name__ == '__main__':
    # 전역 변수로 totalCount를 선언
    global totalCount
    # totalCount를 ThreadVariable 오브젝트로 초기화한다
    totalCount = ThreadVariable()
    # totalCount를 1씩 더하는 
    # Counter Thread를 4개 만들어서 동작시킨다.
    for _ in range(4):
        LockThread = CounterThread()
        LockThread.start()
    
    print('모든 Thread들이 종료될 때까지 기다린다.')
    mainThread = threading.currentThread()
    for thread in threading.enumerate():
        # Main Thread를 제외한 모든 Thread들이 
        # 카운팅을 완료하고 끝날 때 까지 기다린다.
        if thread is not mainThread:
            thread.join()

print('totalCount = ' + str(totalCount.lockedValue))

모든 Thread들이 종료될 때까지 기다린다.
2,500,000번 카운팅 끝!
2,500,000번 카운팅 끝!
2,500,000번 카운팅 끝!2,500,000번 카운팅 끝!



2) 스레드 동기화 처리
    - 위측 코드를 실행하면 threading 모듈의 Lock인스턴스를 생성하고
      Lock클래스에 있는 acquire()로 다른 스레드가 접근하지 못하게
      막고 있으며 작업이 다 끝나면 release()로 다른 스레드의 접근을
      허용한다. 데이터의 신뢰성을 보장.
      이것이 파이썬의 동기화 처리이니 기억해두도록 한다.
      상당히 중요한 내용이다.
      아울러 thread.enumerate()함수는 현재 활성화 되어 있는 
      즉 살아있는 스레드들을 리스트로 반환하는 함수이다.